In [ ]:
import os
import cv2
import glob
import ntpath
import numpy
import pandas

In [ ]:
rate_nodule_attribute()

ndsb_list = os.listdir("G:/LungCancerPredict/extracted/ndsb3_extracted_images/")
res_list = pandas.read_csv("G:/LungCancerPredict/detected/cnn_detect_ndsb3_nms_cube_attribute/res_list.csv")

count=0

for patient_id in ndsb_list:
    
    df = res_list.loc[res_list["patient_id"]==patient_id ]
    
    df.to_csv("G:/LungCancerPredict/detected/cnn_detect_ndsb3_nms_cube_attribute/" + patient_id + ".csv", index=False)
    
    count+=1
    
    print(count)

In [ ]:
def rate_nodule_attribute():
    
    csv_target_path = "G:/LungCancerPredict/detected/cnn_detect_ndsb3_nms_cube_attribute/res_list.csv"

    model_path_texture       = "G:/LungCancerPredict/model/out_texture.hd5"
    model_path_subtlety      = "G:/LungCancerPredict/model/out_subtlety.hd5"
    model_path_spiculation   = "G:/LungCancerPredict/model/out_spiculation.hd5"
    model_path_sphericiy     = "G:/LungCancerPredict/model/out_sphericiy.hd5"
    model_path_margin        = "G:/LungCancerPredict/model/out_margin.hd5"
    model_path_malignancy    = "G:/LungCancerPredict/model/out_malignancy.hd5"
    model_path_lobulation    = "G:/LungCancerPredict/model/out_lobulation.hd5"
    model_path_diamter       = "G:/LungCancerPredict/model/out_diamter.hd5"
    model_path_calcification = "G:/LungCancerPredict/model/out_calcification.hd5"

    model_texture       = get_net(load_weight_path=model_path_texture)
    model_subtlety      = get_net(load_weight_path=model_path_subtlety)
    model_spiculation   = get_net(load_weight_path=model_path_spiculation)
    model_sphericiy     = get_net(load_weight_path=model_path_sphericiy)
    model_margin        = get_net(load_weight_path=model_path_margin)
    model_malignancy    = get_net(load_weight_path=model_path_malignancy)
    model_lobulation    = get_net(load_weight_path=model_path_lobulation)
    model_diamter       = get_net(load_weight_path=model_path_diamter)
    model_calcification = get_net(load_weight_path=model_path_calcification)
    
    samples = glob.glob("G:/LungCancerPredict/detected/cnn_detect_ndsb3_nms_cube/*.png")
    print(len(samples))
    
    #按照batch_size进行预测
    batch_img_list = []; batch_info_list = []; res_list = []
    batch_size = 32; done_count = 0; crop_size = 32

    for index, sample in enumerate(samples):

        print(index)

        file_name = ntpath.basename(sample)

        parts = file_name.split("_")
        
        chance = parts[-3]; z = parts[-4]; y = parts[-5]; x = parts[-6]; patient_id = parts[-8]
        
        batch_info_list.append((patient_id,x,y,z,chance))

        cube_image = load_cube_img(sample, 8, 8, 64)

        indent_x = int((64 - crop_size) / 2)
        indent_y = int((64 - crop_size) / 2)
        indent_z = int((64 - crop_size) / 2)

        cube_image = cube_image[indent_z:indent_z + crop_size, indent_y:indent_y + crop_size, indent_x:indent_x + crop_size]

        img3d = normalize_and_expand(cube_image)

        batch_img_list.append(img3d)
     
        done_count += 1

        if len(batch_img_list)% batch_size==0 or done_count==len(samples):

            batch_data = numpy.vstack(batch_img_list)
            p_diamter = model_diamter.predict(batch_data, batch_size=batch_size)
            p_malignancy = model_malignancy.predict(batch_data, batch_size=batch_size)
            p_spiculation = model_spiculation.predict(batch_data, batch_size=batch_size)
            p_lobulation = model_lobulation.predict(batch_data, batch_size=batch_size)
            p_sphericiy = model_sphericiy.predict(batch_data, batch_size=batch_size)
            p_margin = model_margin.predict(batch_data, batch_size=batch_size)
            p_subtlety = model_subtlety.predict(batch_data, batch_size=batch_size)
            p_texture = model_texture.predict(batch_data, batch_size=batch_size)
            p_calcification = model_calcification.predict(batch_data, batch_size=batch_size)
            
            for i in range(len(p_diamter)):
                
                diamter = str(p_diamter[i][0]);       malignancy = str(p_malignancy[i][0]); spiculation = str(p_spiculation[i][0])
                lobulation = str(p_lobulation[i][0]); sphericiy = str(p_sphericiy[i][0]);   margin = str(p_margin[i][0])
                subtley = str(p_subtlety[i][0]);      textrue = str(p_texture[i][0]);       calcification = str(p_calcification[i][0])

                patient_id=batch_info_list[i][0]; chance = batch_info_list[i][4]
                
                x = batch_info_list[i][1]; y = batch_info_list[i][2]; z = batch_info_list[i][3]
                
                csv_line = [patient_id, x, y, z, chance,diamter,malignancy,spiculation,lobulation,sphericiy,margin,subtley,textrue,calcification]
                
                res_list.append(csv_line)
            
            batch_img_list = []; batch_info_list = []
    
    df = pandas.DataFrame(res_list, columns=["patient_id", "coord_x", "coord_y", "coord_z", "nodule_chance","diamter","malignancy","spiculation","lobulation","sphericiy","margin","subtley","textrue","calcification"])
    df.to_csv(csv_target_path, index=False)



def load_cube_img(src_path, rows, cols, size):
    
    img = cv2.imread(src_path, cv2.IMREAD_GRAYSCALE)
    res = numpy.zeros((rows * cols, size, size))

    img_height = size
    img_width = size

    for row in range(rows):
        
        for col in range(cols):
            
            src_y = row * img_height
            src_x = col * img_width
            res[row * cols + col] = img[src_y:src_y + img_height, src_x:src_x + img_width]

    return res

def normalize_and_expand(img):
    
    img = img.astype(numpy.float32)
    img -= 41  # 41 is MEAN_PIXEL_VALUE
    img /= 255.
    img = img.reshape(1, img.shape[0], img.shape[1], img.shape[2], 1)
    
    return img

In [ ]:
from keras.models import Model
from keras.optimizers import SGD 
from keras.metrics import mean_absolute_error
from keras.layers import Input, AveragePooling3D, Convolution3D, MaxPooling3D, Flatten

def get_net(input_shape=(32, 32, 32, 1), load_weight_path=None) -> Model: 
    
    inputs = Input(shape=input_shape, name="input_1")
    x = inputs
    x = AveragePooling3D(pool_size=(2, 1, 1), strides=(2, 1, 1), border_mode="same")(x)
    x = Convolution3D(64, 3, 3, 3, activation='relu', border_mode='same', name='conv1', subsample=(1, 1, 1))(x)
    x = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), border_mode='valid', name='pool1')(x)

    # 2nd layer group
    x = Convolution3D(128, 3, 3, 3, activation='relu', border_mode='same', name='conv2', subsample=(1, 1, 1))(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), border_mode='valid', name='pool2')(x)

    # 3rd layer group
    x = Convolution3D(256, 3, 3, 3, activation='relu', border_mode='same', name='conv3a', subsample=(1, 1, 1))(x)
    x = Convolution3D(256, 3, 3, 3, activation='relu', border_mode='same', name='conv3b', subsample=(1, 1, 1))(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), border_mode='valid', name='pool3')(x)

    # 4th layer group
    x = Convolution3D(512, 3, 3, 3, activation='relu', border_mode='same', name='conv4a', subsample=(1, 1, 1))(x)
    x = Convolution3D(512, 3, 3, 3, activation='relu', border_mode='same', name='conv4b', subsample=(1, 1, 1),)(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), border_mode='valid', name='pool4')(x)

    # output
    last64 = Convolution3D(64, 2, 2, 2, activation="relu", name="last_64")(x)
    out_attribute = Convolution3D(1, 1, 1, 1, activation=None, name="out_attribute_last")(last64)
    out_attribute = Flatten(name="out_texture")(out_attribute)

    model = Model(input=inputs, output=out_attribute)
    
    if load_weight_path is not None: model.load_weights(load_weight_path, by_name=False)

    optimizer = SGD(lr=0.001, momentum=0.9, nesterov=True)
    loss = {"out_texture": "mean_absolute_error"}
    metrics = {"out_texture": [mean_absolute_error]}

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    model.summary(line_length=140)

    return model